In [1]:
import numpy as np
import pandas as pd
import spacy
import re
from itertools import chain

In [2]:
dat = pd.read_pickle("../data/es_qa_gcloud_final.pkl")

In [3]:
nlp = spacy.load("es")

In [4]:
with open("../wordvecs/wiki.es/wiki.es.nospace.vec") as f:
    f.readline()
    nlp.vocab.load_vectors(f)

In [5]:
dat['question_nlp'] = dat['question'].apply(lambda x: nlp(x, parse=False, tag=False, entity=False))
dat['story_nlp'] = dat['story'].apply(lambda x: nlp(x, parse=False, tag=False, entity=False))

In [6]:
dat.head()

,answer,question,story,question_nlp,story_nlp
0,@entity2,La información proviene de los documentos del...,La @entity0 ( @entity0) trabaja en la construc...,"( , La, información, proviene, de, los, docume...","(La, @entity0, (, @entity0, ), trabaja, en, la..."
1,@entity10,@placeholder y su marido fueron asesinados en...,Esta semana @entity12 ha recibido una noticia ...,"( , @placeholder, y, su, marido, fueron, asesi...","(Esta, semana, @entity12, ha, recibido, una, n..."
2,@entity1,Han detenido a 7 personas intregrantes de la ...,Esta semana @entity12 ha recibido una noticia ...,"( , Han, detenido, a, 7, personas, intregrante...","(Esta, semana, @entity12, ha, recibido, una, n..."
3,@entity12,@placeholder tiene una gran tasa de criminali...,Esta semana @entity12 ha recibido una noticia ...,"( , @placeholder, tiene, una, gran, tasa, de, ...","(Esta, semana, @entity12, ha, recibido, una, n..."
4,@entity3,"Su líder moral, el dr. @placeholder, dice en ...",La situación en el Estado @entity14 de @entity...,"( , Su, líder, moral, ,, el, dr, ., @placehold...","(La, situación, en, el, Estado, @entity14, de,..."


In [7]:
dat.shape

(78928, 5)

In [8]:
q_lengths = dat['question_nlp'].apply(len)
s_lengths = dat['story_nlp'].apply(len)

In [9]:
# question_has_vec_ind = dat['question_nlp'].apply(lambda x: np.array([w.has_vector for w in x]))

In [10]:
# story_has_vec_ind = dat['story_nlp'].apply(lambda x: np.array([w.has_vector for w in x]))

In [11]:
question_ent_ind = dat['question_nlp'].apply(lambda x: np.array([w.text[:7] == '@entity' for w in x]))
story_ent_ind = dat['story_nlp'].apply(lambda x: np.array([w.text[:7] == '@entity' for w in x]))

In [12]:
question_ph_ind = dat['question_nlp'].apply(lambda x: np.array([w.text == '@placeholder' for w in x]))

In [13]:
no_ph_ind = (question_ph_ind.apply(sum) == 0)

In [14]:
no_ans_ind = []
for i in range(dat.shape[0]):
    no_ans_ind.append(re.search(dat['answer'][i], dat['story'][i]) is None)
no_ans_ind = np.array(no_ans_ind)

In [15]:
selected = dat.loc[(q_lengths > 4) & (q_lengths <= 50) & (s_lengths <= 2000) & (~no_ans_ind) & (~(no_ph_ind))]

In [16]:
selected.shape

(76775, 5)

In [17]:
selected.head()

,answer,question,story,question_nlp,story_nlp
0,@entity2,La información proviene de los documentos del...,La @entity0 ( @entity0) trabaja en la construc...,"( , La, información, proviene, de, los, docume...","(La, @entity0, (, @entity0, ), trabaja, en, la..."
1,@entity10,@placeholder y su marido fueron asesinados en...,Esta semana @entity12 ha recibido una noticia ...,"( , @placeholder, y, su, marido, fueron, asesi...","(Esta, semana, @entity12, ha, recibido, una, n..."
2,@entity1,Han detenido a 7 personas intregrantes de la ...,Esta semana @entity12 ha recibido una noticia ...,"( , Han, detenido, a, 7, personas, intregrante...","(Esta, semana, @entity12, ha, recibido, una, n..."
3,@entity12,@placeholder tiene una gran tasa de criminali...,Esta semana @entity12 ha recibido una noticia ...,"( , @placeholder, tiene, una, gran, tasa, de, ...","(Esta, semana, @entity12, ha, recibido, una, n..."
4,@entity3,"Su líder moral, el dr. @placeholder, dice en ...",La situación en el Estado @entity14 de @entity...,"( , Su, líder, moral, ,, el, dr, ., @placehold...","(La, situación, en, el, Estado, @entity14, de,..."


In [18]:
sel = selected.drop(['story_nlp', 'question_nlp'], axis=1)

In [19]:
sel.head()

,answer,question,story
0,@entity2,La información proviene de los documentos del...,La @entity0 ( @entity0) trabaja en la construc...
1,@entity10,@placeholder y su marido fueron asesinados en...,Esta semana @entity12 ha recibido una noticia ...
2,@entity1,Han detenido a 7 personas intregrantes de la ...,Esta semana @entity12 ha recibido una noticia ...
3,@entity12,@placeholder tiene una gran tasa de criminali...,Esta semana @entity12 ha recibido una noticia ...
4,@entity3,"Su líder moral, el dr. @placeholder, dice en ...",La situación en el Estado @entity14 de @entity...


In [20]:
# re.sub(r'(?<=\S)@entity(\d+)', r' @entity\1', '@entity23,@entity67@entity56')

In [21]:
# deliminated_questions = sel['question'].apply(lambda x: re.sub(r'(?<=\S)@entity(\d+)', r' @entity\1', x))
# deliminated_stories = sel['story'].apply(lambda x: re.sub(r'(?<=\S)@entity(\d+)', r' @entity\1', x))

In [22]:
# sel['question'] = deliminated_questions
# sel['story'] = deliminated_stories

In [23]:
dev = sel.sample(frac=0.1, random_state=7777, replace=False)
rest = sel[~np.in1d(sel.index, dev.index)]
test = rest.sample(frac=0.1, random_state=7777, replace=False)
train = rest[~np.in1d(rest.index, test.index)]

In [24]:
print(dev.shape, rest.shape, test.shape, train.shape)

(7678, 3) (69097, 3) (6910, 3) (62187, 3)


In [25]:
train.to_pickle("../input_data/train_es.pkl")
dev.to_pickle("../input_data/dev_es.pkl")
test.to_pickle("../input_data/test_es.pkl")